In [117]:
%reload_ext autoreload
%autoreload 2

import os
import json
import glob
# from semproc.parser import Parser
from semproc.identifier import Identify
from semproc.rawresponse import RawResponse

In [6]:
def _prep(filepath):
    with open(filepath, 'r') as f:
        response = f.read()

    response = response.replace('\\\n', '').replace('\r\n', '').replace('\\r', '').replace('\\n', '').replace('\n', '')
    return response.decode('utf-8', errors='replace').encode('unicode_escape')

In [7]:
files = [
    ('http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities', 
     '../response_examples/cwic_csw_v2_0_2.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_iso.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_dif.xml'),
    ('http://www.data.gov?service=csw&version=2.0.2&request=getrecords', 
     '../response_examples/datagov_csw_202_getrecords_fgdc.xml'),
    ('http://www.data.gov/this/is/rdf', 
     '../response_examples/datagov_9bcffa1c-6164-4635-bc2c-6c98cce59d7b.rdf'),
    ('http://www.data.gov/iso/ds', 
     '../response_examples/iso-19115_ds.xml'),
    ('http://www.data.gov/iso/md', 
     '../response_examples/iso-19115_md.xml'),
    ('http://www.data.gov/iso/mi', 
     '../response_examples/iso-19115_mi.xml'),
    ('http://www.data.gov/iso/srv', 
     '../response_examples/iso-19119_as_md.xml'),
    ('http://www.example.gov/oai', 
     '../response_examples/oaipmh_listrecords.xml'),
    ('http://www.example.gov/oai', 
     '../response_examples/oaipmh_identify.xml'),
    ('http://www.example.gov/opendap/wsdl', 
     '../response_examples/opendap_wsdl.xml'),
    ('http://www.example.gov/os/osdd.xml', 
     '../response_examples/opensearch_blended_parameters.xml'),
    ('http://www.example.gov/os/atom', 
     '../response_examples/opensearch_dspace_search_atom.xml'),
    ('http://www.example.gov/or/rss', 
     '../response_examples/opensearch_ngdc_search_rss.xml'),
    ('http://www.example.gov/sos?service=sos&version=1.0.0&request=getcapabilities', 
     '../response_examples/sos_v1_0_0.xml'),
    ('http://www.example.gov/thredds/eol', 
     '../response_examples/thredds_catalog_eol.xml'),
    ('http://www.example.gov/thredds/catalog', 
     '../response_examples/thredds_catalog.xml'),
    ('http://www.data.gov?service=wcs&version=1.0.0&request=describecoverage', 
     '../response_examples/wcs_v1_0_0_describe_coverage.xml'),
    ('http://www.data.gov?service=wcs&version=1.0.0&request=getcapabilities', 
     '../response_examples/wcs_v1_0_0.xml'),
    ('http://www.data.gov?service=wcs&version=1.1.0&request=getcapabilities', 
     '../response_examples/wcs_v1_1_0.xml'),
    ('http://www.data.gov?service=wcs&version=1.1.2&request=getcapabilities', 
     '../response_examples/wcs_v1_1_2.xml'),
    ('http://www.data.gov?service=wfs&version=1.0.0&request=getcapabilities', 
     '../response_examples/wfs_v1_0_0.xml'),
    ('http://www.data.gov?service=wfs&version=1.1.0&request=getcapabilities', 
     '../response_examples/wfs_v1_1_0.xml'),
    ('http://www.data.gov?service=wms&version=1.3.0&request=getcapabilities', 
     '../response_examples/wms_v1_3_0.xml'),
    ('http://www.data.gov?service=wms&version=1.1.1&request=getcapabilities', 
     '../response_examples/wms_v1_1_1.xml'),
    ('http://www.data.gov?service=wms&version=1.3.0&request=getcapabilities', 
     '../response_examples/wms_v1.3.0.xml')
]

In [118]:
yamls = glob.glob('../semproc/configs/ogc_identifier.yaml')

for url, filepath in files[:1]:
    response = _prep(filepath)
    
    i = Identify(yamls, response, url)
    matches = i.identify()
    
    print
    print 'Identification----------------'
    print '\t', url, filepath
    print matches
    


Identification----------------
	http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities ../response_examples/cwic_csw_v2_0_2.xml
[{'protocol': 'OGC', 'service': {'name': 'CSW', 'request': 'GetCapabilities'}}]


In [115]:
import yaml
import re
# y = yaml.load("""
# - name: SOS
#   request: GetCapabilities
#   filters:
#     ands:

#       - type: simple
#         object: content
#         value: 'http://www.opengis.net/sos'
#       - type: simple
#         object: content
#         value: 'Operation name="DescribeSensor"'

#       - type: complex
#         operator: ors
#         filters:
#           - type: simple
#             object: url
#             value: 'REQUEST=GETCAPABILITIES'
#           - type: simple
#             object: content
#             value: '<GetCapabilities>'
#           - type: simple
#             object: content
#             value: 'name="GetCapabilities"'""")

with open('../semproc/configs/test_ogc.yaml', 'r') as f:
    y = yaml.load(f.read())


response = _prep('../response_examples/cwic_csw_v2_0_2.xml')
url = 'http://cwic.nasa.gov/something?service=csw&version=2.0.2&request=getcapabilities'
xml = etree.fromstring(response)

def _filter(xml, source_content, source_url, operator, filters, clauses):
    '''
    generate a list of dicts for operator and booleans
    that can be rolled up into some bool for a match
    '''
    for f in filters:
        filter_type = f['type']

        if filter_type == 'complex':
            filter_operator = f['operator']
            clauses.append(_filter(xml, source_content, source_url, filter_operator, f['filters'], []))
            print 'COMPLEX', filter_operator
        elif filter_type == 'simple':
            filter_object = source_content if f['object'] == 'content' else source_url
            filter_value = f['value']

            # TODO: a better solution than this
            filter_value = filter_value.upper()
            filter_object = filter_object.upper()

            clauses.append(filter_value in filter_object)
            
            print 'SIMPLE', f['value'], filter_value in filter_object
        elif filter_type == 'regex':
            filter_object = source_content if f['object'] == 'content' else source_url
            filter_value = f['value']
            clauses.append(len(re.findall(filter_value, filter_object)) > 0)
            
            print 'regex', f['value'], len(re.findall(filter_value, filter_object)) > 0
        elif filter_type == 'xpath':
            # if the filter is xpath, we can only run against
            # the provided xml (parser) and ONLY evaluate for existence
            # ie the xpath returned some element, list, text value
            # but we don't care what it returned
            xpath = f['value']
            if xml is None:
                # nothing to find, this is an incorrect filter
                clauses.append(False)

            # try the xpath but there could be namespace or
            # other issues (also false negatives!)
            try:
                clause = xml.xpath(xpath) not in [None, '', []]
            except:
                clause = False

            print 'XPATH', f['value'], clause
            clauses.append(clause)

    print 'filtered: ', {operator: clauses}
    return {operator: clauses}

def _evaluate(clauses, sums):
    '''
    evaluate a list a dicts where the key is
    the operator and the value is a list of
    booleans
    '''
    if isinstance(clauses, bool):
        # so this should be the rolled up value
        return clauses

    for k, v in clauses.iteritems():
        if isinstance(v, dict):
            e = _evaluate(v, 0)
            print 'dict', k, e, sums
            return sums + e
        elif isinstance(v, list) and not all(isinstance(i, bool) for i in v):
            # TODO: this is not a good assumption
            print 'i am a list', v
            new_list = []
            for i in v:
                new_list.append(_evaluate(i, 0))
            #return sums
            v = new_list
        if k == 'ands':
            # everything must be true
            sums += sum(v) == len(v)
            print 'ands = ', sum(v) == len(v), v
        elif k == 'ors':
            # any one must be true
            sums += sum(v) > 0
            print 'ors = ', sum(v) > 0, v

    return sums


# so it's not respecting the parent *and* instead 
# running the or and we are getting false matches on the getcapabilities flags.
    



In [116]:
for x in y:
    for k, v in x.iteritems():
        if k == 'name' or v is None:
            continue

        for option in v:
            filters = option['filters']

            for i, j in filters.iteritems():
                #print i, j
                final_sums = _evaluate({i: _filter(xml, response, url, i, j, [])}, 0)
                print '----------------------------'
                print 'returned: ', final_sums
                if final_sums:
                    print 'MATCHED: ', x['name'], option['name']

        

SIMPLE GetRecordsResponse False
SIMPLE http://www.opengis.net/cat/csw/ True
filtered:  {'ands': [False, True]}
ands =  False [False, True]
dict ands 0 0
----------------------------
returned:  0
SIMPLE http://www.opengis.net/wms False
SIMPLE <!DOCTYPE WMT_MS_Capabilities SYSTEM "http://schemas.opengis.net/wms/1.1.1/WMS_MS_Capabilities.dtd" False
filtered:  {'ors': [False, False]}
COMPLEX ors
SIMPLE REQUEST=GETCAPABILITIES True
SIMPLE <GetCapabilities> False
SIMPLE name="GetCapabilities" True
filtered:  {'ors': [True, False, True]}
COMPLEX ors
filtered:  {'ands': [{'ors': [False, False]}, {'ors': [True, False, True]}]}
i am a list [{'ors': [False, False]}, {'ors': [True, False, True]}]
ors =  False [False, False]
ors =  True [True, False, True]
ands =  False [0, 1]
dict ands 0 0
----------------------------
returned:  0
SIMPLE http://www.opengis.net/cat/csw True
SIMPLE SERVICE=CSW True
SIMPLE <csw:Capabilities True
filtered:  {'ors': [True, True]}
COMPLEX ors
SIMPLE REQUEST=GETCAPABILIT